In [50]:
import pandas as pd
import numpy as np
from sklearn.decomposition import pca
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
norm_data = pd.read_csv("../csv/expanded_dataset_norm_v1.csv")
data = norm_data.drop(['Class'], axis=1)
target = norm_data['Class']
norm_data = norm_data.drop(norm_data.columns[[0,]], axis=1)

In [136]:
#one class svm normalized data
from sklearn import svm

svc1 = svm.OneClassSVM(kernel="rbf", gamma=2 ** 9, nu = 0.1)

inliers = norm_data.loc[norm_data['Class'] == 'normal',:].drop(['Class'], axis=1)
outliers_flood = norm_data.loc[norm_data['Class'] == 'flooding', :].drop(['Class'], axis=1)
outliers_slow = norm_data.loc[norm_data['Class'] == 'slowloris', :].drop(['Class'], axis=1)

target_in = np.ones(len(inliers.index))
target_out_flood = np.full(len(outliers_flood.index), -1.0)
target_out_slow = np.full(len(outliers_slow.index), -1.0)
target_all = np.concatenate([target_in, target_out_flood, target_out_slow])


svc1.fit(inliers)
# predicted_in = svc1.predict(inliers)
predicted_out_flood = svc1.predict(outliers_flood)
predicted_out_slow = svc1.predict(outliers_slow)
# predicted_all = np.concatenate([predicted_in, predicted_out_flood, predicted_out_slow])

# ac = metrics.accuracy_score(target_all, predicted_all)
# ac_in = metrics.accuracy_score(target_in, predicted_in)
ac_out_flood = metrics.accuracy_score(target_out_flood, predicted_out_flood)
ac_out_slow = metrics.accuracy_score(target_out_slow, predicted_out_slow)

print(ac_out_flood, ac_out_slow)
predicted = cross_val_predict(svc1, inliers, target_in, cv=10)
metrics.accuracy_score(predicted, target_in)

(0.73359073359073357, 0.81725888324873097)


0.57597173144876324

In [120]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(inliers)
p = np.empty(1, dtype=int)
t = np.empty(1, dtype=int)
svc1 = svm.OneClassSVM(kernel="rbf", gamma=5, nu = 0.0001)
for train_index, test_index in kf.split(norm_data):
    
    train_data = norm_data.iloc[train_index]
    test_data = norm_data.iloc[test_index]
    
    inliers_train = train_data.loc[norm_data['Class'] == 'normal', :].drop(['Class'], axis=1)
    
#     outliers_flood = test_data.loc[norm_data['Class'] == 'flooding', :].drop(['Class'], axis=1)
#     outliers_slow = test_data.loc[norm_data['Class'] == 'slowloris', :].drop(['Class'], axis=1)
    inliers_test = test_data.loc[norm_data['Class'] == 'normal', :].drop(['Class'], axis=1)
    
    t1 = np.ones(len(inliers_test.index))
#     t2 = np.full(len(outliers_slow.index), -1.0)
#     t3 = np.full(len(outliers_flood.index), -1.0)
    
    t = np.append(t, t1)
#     t = np.append(t, t2)
#     t = np.append(t, t3)
    
    svc1.fit(inliers_train)
    
    p1 = svc1.predict(inliers_test)
#     p2 = svc1.predict(outliers_slow)
#     p3 = svc1.predict(outliers_flood)
    
    p = np.append(p, p1)
#     p = np.append(p, p2)
#     p = np.append(p, p3)
    
metrics.accuracy_score(p, t)

0.89947089947089942

[  5.05429156e-321   1.00000000e+000   1.00000000e+000 ...,
   1.00000000e+000   1.00000000e+000  -1.00000000e+000]
